### Most frequent n-word combinations
    - you can just change the number desired_number, 
    but the program won't work unless you have the condensed_files.txt file in the same folder

In [4]:
import operator
import itertools
import sys

f = open('condensed_files.txt','r')
s = f.readlines()
metrics_per_study = [x.replace('\t',';').replace('\n','').replace(' ','_').replace('#','').lower().split(';')
      for x in s if not (x.startswith('==>' ) or x.startswith('\n' ))]

def get_all(x):
    for i in x:
        for j in i:
            yield(j)

x = [j for j in get_all(metrics_per_study)]

wordcount={}
for word in x:
    if word not in wordcount:
        wordcount[word] = 1
    else:
        wordcount[word] += 1


best_words = sorted(wordcount.items(), key=operator.itemgetter(1),reverse=True)[0:25]

def get_best_combo(n):
    n_word_combos = list(itertools.combinations(range(len(best_words)), n))
    words_frequencies={}

    for i in n_word_combos:
        words = []
        for k in range (n):
            words.append(best_words[i[k]][0])
        word_string = ''
        for l in words:
            word_string += l
            word_string += '    '
        word_string=word_string.rstrip("    ")
        for j in metrics_per_study:
            if set(words).issubset(set(j)):
                if word_string in words_frequencies:
                    words_frequencies[word_string] += 1
                else:
                    words_frequencies[word_string] =1
    for key, value in words_frequencies.items():
        words_frequencies[key] = round(value / len(metrics_per_study),2)

    sorted_combo_frequencies = sorted(words_frequencies.items(), key=operator.itemgetter(1),reverse=True)
    return sorted_combo_frequencies



# just call this to see how often a combination of attribute names appear in studies
def print_best(desired_number):
    print("This is a dictionary where:")
    print("-Key is the combination of attributes")
    print("-Value is the frequency of that combination of attributes in attributes of medical studies data")
    print()
    j = 0
    for i in get_best_combo(int(desired_number)):
        if (j < 10):
            j+=1
            print(i)
    return

In [5]:
print_best(1)

This is a dictionary where:
-Key is the combination of attributes
-Value is the frequency of that combination of attributes in attributes of medical studies data

('cancer_type', 0.8)
('patient_identifier', 0.78)
('cancer_type_detailed', 0.78)
('sample_identifier', 0.78)
('diagnosis_age', 0.52)
('oncotree_code', 0.42)
('person_gender', 0.35)
('overall_survival_status', 0.32)
('overall_survival_(months)', 0.27)
('primary_tumor_site', 0.22)


In [6]:
print_best(2)

This is a dictionary where:
-Key is the combination of attributes
-Value is the frequency of that combination of attributes in attributes of medical studies data

('cancer_type    patient_identifier', 0.78)
('cancer_type_detailed    patient_identifier', 0.78)
('cancer_type    sample_identifier', 0.78)
('patient_identifier    sample_identifier', 0.78)
('cancer_type    cancer_type_detailed', 0.78)
('cancer_type_detailed    sample_identifier', 0.78)
('patient_identifier    diagnosis_age', 0.52)
('cancer_type    diagnosis_age', 0.52)
('cancer_type_detailed    diagnosis_age', 0.52)
('sample_identifier    diagnosis_age', 0.52)


In [7]:
print_best(3)

This is a dictionary where:
-Key is the combination of attributes
-Value is the frequency of that combination of attributes in attributes of medical studies data

('cancer_type_detailed    patient_identifier    sample_identifier', 0.78)
('cancer_type    patient_identifier    sample_identifier', 0.78)
('cancer_type    cancer_type_detailed    patient_identifier', 0.78)
('cancer_type    cancer_type_detailed    sample_identifier', 0.78)
('patient_identifier    sample_identifier    diagnosis_age', 0.52)
('cancer_type_detailed    sample_identifier    diagnosis_age', 0.52)
('cancer_type    sample_identifier    diagnosis_age', 0.52)
('cancer_type    patient_identifier    diagnosis_age', 0.52)
('cancer_type    cancer_type_detailed    diagnosis_age', 0.52)
('cancer_type_detailed    patient_identifier    diagnosis_age', 0.52)


In [8]:
print_best(4)

This is a dictionary where:
-Key is the combination of attributes
-Value is the frequency of that combination of attributes in attributes of medical studies data

('cancer_type    cancer_type_detailed    patient_identifier    sample_identifier', 0.78)
('cancer_type    patient_identifier    sample_identifier    diagnosis_age', 0.52)
('cancer_type_detailed    patient_identifier    sample_identifier    diagnosis_age', 0.52)
('cancer_type    cancer_type_detailed    sample_identifier    diagnosis_age', 0.52)
('cancer_type    cancer_type_detailed    patient_identifier    diagnosis_age', 0.52)
('cancer_type    cancer_type_detailed    patient_identifier    person_gender', 0.35)
('cancer_type    cancer_type_detailed    sample_identifier    person_gender', 0.35)
('cancer_type    patient_identifier    sample_identifier    person_gender', 0.35)
('cancer_type_detailed    patient_identifier    sample_identifier    person_gender', 0.35)
('cancer_type    patient_identifier    sample_identifier    over

In [9]:
print_best(5)

This is a dictionary where:
-Key is the combination of attributes
-Value is the frequency of that combination of attributes in attributes of medical studies data

('cancer_type    cancer_type_detailed    patient_identifier    sample_identifier    diagnosis_age', 0.52)
('cancer_type    cancer_type_detailed    patient_identifier    sample_identifier    person_gender', 0.35)
('cancer_type    cancer_type_detailed    patient_identifier    sample_identifier    overall_survival_status', 0.32)
('cancer_type_detailed    patient_identifier    sample_identifier    diagnosis_age    person_gender', 0.3)
('cancer_type    cancer_type_detailed    sample_identifier    diagnosis_age    person_gender', 0.3)
('cancer_type    patient_identifier    sample_identifier    diagnosis_age    person_gender', 0.3)
('cancer_type    cancer_type_detailed    patient_identifier    diagnosis_age    person_gender', 0.3)
('cancer_type_detailed    patient_identifier    sample_identifier    overall_survival_status    overall

### Measuring similiarity of two studies:

###### Gives the similiarity of two studies based on:
###### a) attribute names
###### b) attribute values (not very refined)

In [14]:
import os
import pandas as pd

# condensed_files has a) file location b) file headers
f = open('condensed_files.txt','r')
s = f.readlines()

study_files = [x.replace('==>','').replace('<==\n','').replace(' ','').replace('./','/')
      for x in s if (x.startswith('==>' ) )]



path = os.getcwd().replace('/script_project','')


def read_file(filepath):
    k = open(filepath)
    num_rows = 0
    j=k.readline()
    while j.startswith('#'):
        j = k.readline()
        num_rows+=1
    return pd.read_csv(filepath, sep = '\t', skiprows=num_rows, header='infer')

def study_similiarity(study1:pd.core.frame.DataFrame, study2:pd.core.frame.DataFrame):
    study1_attrs=set(study1.columns.values)
    study2_attrs=set(study2.columns.values)
    same_attributes = study1_attrs.intersection(study2_attrs)
    unique_attrs_study1 = study1_attrs-same_attributes
    unique_attrs_study2 = study2_attrs-same_attributes
    
    similar_attributes = ''
    for i in study1.columns:
        study1_unique = set(study1[i].unique())
        for j in study2.columns:
            study2_unique = set(study2[j].unique())
            if len(study1_unique.intersection(study2_unique)) > (0.75 * min(len(study1_unique), len(study2_unique))):
                similar_attributes += (i +' with ' + j + ', ')
            
    print("Study1 attributes:", study1_attrs)
    print()
    print("Study2 attributes:", study2_attrs)
    print("-----------------------------------------------------------------------------------------\n")
    print("Same attributes:",same_attributes)
    #print("-------------------------------------\n")
    print("Similar attributes:",similar_attributes)
    return 

In [15]:
df1 = read_file(path+study_files[21])
df2 = read_file(path+study_files[35])

study_similiarity(df1,df2)

Study1 attributes: {'ABERRANT_CELL_FRACTION', 'SAMPLE_ID', 'MITOTIC_COUNT', 'MITOTIC_SCORE', 'PATIENT_ID', 'TUBULE_SCORE', 'PLEOMORPHISM_SCORE', 'HER2_STATUS', 'TUMOR_PLOIDY', 'ER_STATUS', 'AGE', 'PR_STATUS', 'HISTOPATHOLOGICAL_DIAGNOSIS', 'GRADE', 'CANCER_TYPE_DETAILED', 'CANCER_TYPE'}

Study2 attributes: {'OS_STATUS', 'PRIMARY_SITE', 'SAMPLE_ID', 'MRNA_DATA', 'PATIENT_ID', 'PRIMARY_TUMOR_PATHOLOGIC_SPREAD', 'OS_MONTHS', 'SEQUENCED', 'MLH1_SILENCING', 'GENDER', 'METHYLATION', 'EXPRESSION_SUBTYPE', 'MSI_STATUS', 'HISTOLOGICAL_SUBTYPE', 'COPY_NUMBER', 'ICLUSTER', 'HYPERMUTATED', 'CANCER_TYPE_DETAILED', 'METHYLATION_SUBTYPE', 'TUMOR_STAGE_2009', 'CANCER_TYPE'}
-----------------------------------------------------------------------------------------

Same attributes: {'PATIENT_ID', 'SAMPLE_ID', 'CANCER_TYPE_DETAILED', 'CANCER_TYPE'}
Similar attributes: MITOTIC_COUNT with SEQUENCED, MITOTIC_COUNT with COPY_NUMBER, MITOTIC_COUNT with MRNA_DATA, MITOTIC_COUNT with METHYLATION, 


In [16]:
df1 = read_file(path+study_files[12])
df2 = read_file(path+study_files[32])

study_similiarity(df1,df2)

Study1 attributes: {'PRIMARY_SITE', 'SAMPLE_ID', 'SAMPLE_DISPLAY_NAME', 'OS_MONTHS', 'AGE', 'CANCER_TYPE_DETAILED', 'DFS_MONTHS', 'SAMPLE_CLASS', 'SITE_OF_RECURRENCE', 'YEAR_OF_SURGERY', 'ONCOTREE_CODE', 'DFS_STATUS', 'LYMPH_NODE_STATUS', 'CANCER_TYPE', 'PATIENT_DISPLAY_NAME', 'GENE_PANEL', 'SEX', 'TNMSTAGE', 'TUMOR_PATH_STAGE', 'OS_STATUS', 'CLINICAL_STATUS', 'PATIENT_ID', 'SPECIMEN_PRESERVATION_TYPE', 'SAMPLE_TYPE', 'METASTATIC_SITE'}

Study2 attributes: {'TISSUE_SOURCE_SITE', 'SAMPLE_ID', 'PATIENT_ID', 'TUMOR_TYPE', 'STR_STATUS', 'DIAGNOSIS', 'MSI_STATUS', 'TUMOR_CONTENT', 'CANCER_TYPE_DETAILED', 'CANCER_TYPE'}
-----------------------------------------------------------------------------------------

Same attributes: {'PATIENT_ID', 'SAMPLE_ID', 'CANCER_TYPE_DETAILED', 'CANCER_TYPE'}
Similar attributes: SAMPLE_CLASS with TISSUE_SOURCE_SITE, 


In [17]:
df1 = read_file(path+study_files[2])
df2 = read_file(path+study_files[42])

study_similiarity(df1,df2)

Study1 attributes: {'TUMOR_GRADE', 'CANCER_TYPE', 'PRIMARY_SITE', 'SAMPLE_ID', 'GENOMIC_ALTERATIONS', 'ONCOTREE_CODE', 'PATIENT_ID', 'METASTATIC_TUMOR_INDICATOR', 'PLATFORM', 'GENDER', 'TUMOR_TISSUE_SITE', 'AGE', 'CANCER_TYPE_DETAILED', 'SAMPLE_TYPE', 'TUMOR_STAGE', 'ACTIONABLE_ALTERATIONS', 'METASTATIC_SITE'}

Study2 attributes: {'SMOKING_PACK_YEARS', 'SAMPLE_ID', 'PATIENT_ID', 'SMOKER', 'PATH_T_STAGE', 'BARRETTS_ESOPHAGUS', 'SEX', 'PATH_N_STAGE', 'AGE', 'PRIMARY_SITE_OTHER', 'CLIN_M_STAGE', 'ETHNICITY', 'GRADE', 'NONSILENT_PER_MB', 'CANCER_TYPE_DETAILED', 'TUMOR_SITE', 'CANCER_TYPE'}
-----------------------------------------------------------------------------------------

Same attributes: {'PATIENT_ID', 'CANCER_TYPE', 'SAMPLE_ID', 'AGE', 'CANCER_TYPE_DETAILED'}
Similar attributes: AGE with AGE, GENDER with SEX, GENOMIC_ALTERATIONS with PATH_N_STAGE, GENOMIC_ALTERATIONS with CLIN_M_STAGE, ACTIONABLE_ALTERATIONS with PATH_N_STAGE, ACTIONABLE_ALTERATIONS with CLIN_M_STAGE, 


In [18]:
df1 = read_file(path+study_files[17])
df2 = read_file(path+study_files[29])

study_similiarity(df1,df2)

Study1 attributes: {'CANCER_TYPE', 'SAMPLE_ID', 'SEQUENCING_TYPE', 'PATIENT_ID', 'CANCER_TYPE_DETAILED', 'SAMPLE_CLASS'}

Study2 attributes: {'PATIENT_ID', 'SAMPLE_ID', 'ONCOTREE_CODE'}
-----------------------------------------------------------------------------------------

Same attributes: {'PATIENT_ID', 'SAMPLE_ID'}
Similar attributes: 
